# Simulating observations

This notebook shows how to use the `Simulator` class to easily generate synthetic observation datasets, with:
* how to set up the observed environment (using the distributions implemented in `infobs.sampling`)
* how to simulate noise-free observations of the considered environment (using the `IdealInstrument` class)
* how to simulate realistic IRAM-30m observations of the considered environment (using the `IRAM30mEMIR` class)
* how to modify the noise model (eg, adding a beam dilution or changing the integration time)

In [1]:
import os
import sys

sys.path.insert(0, os.path.join(".."))

from infobs import Simulator
from infobs.instruments import IdealInstrument, IRAM30mEMIR
from infobs.sampling import Constant, LogUniform

## Settings

In [2]:
lines = ["13c_o_j1__j0", "13c_o_j2__j1", "13c_o_j3__j2"]

n_samples = 5
linewidth = 10 # km/s
obstime = 1 # Minutes

## Samplers

In [3]:
samplers = {
    'Av': LogUniform(1, 25),
    'G0': LogUniform(1e1, 1e3),
    'Pth': LogUniform(1e5, 5e6),
    'angle': Constant(0.)
}

**Warning:** Sampler intervals should be included in the emulator's range of validity, i.e.,
- $A_v^{\text{max}} \in \left[1,\,40\right]$,
- $G_0 \in \left[1,\,10^5\right]$,
- $P_{\text{th}} \in \left[10^5,\,10^9\right]$,
- $\alpha \in \left[0,\,60\right]$.

The chosen sampler corresponds roughly to the range of values within the horsehead pillar. Note that here a loguniform distribution is used while in practice the distributions are more sophisticated (see `sampling.ipynb` example notebook).

What's more, here the sampler chosen for the angle $\alpha$ is constant at 0, meaning that the PDR is assumed to be always observed face-on.

## Ideal instrument

In [4]:
ideal = IdealInstrument()
simulator = Simulator(ideal)

simulator.simulate(
    n_samples,
    samplers,
    lines,
    obstime
)

,Av,G0,Pth,angle,kappa,13c_o_j1__j0,13c_o_j2__j1,13c_o_j3__j2
0,7.198038,570.499386,3.693990e+06,0.0,1.0,14.992116,28.134336,23.984704
1,4.608825,92.881820,1.439909e+05,0.0,1.0,9.052731,11.887270,4.681478
2,6.942497,185.331395,4.777970e+05,0.0,1.0,15.412903,22.866178,13.715204
3,1.895651,481.791933,4.189151e+06,0.0,1.0,0.251140,0.841297,1.259055
4,21.184942,145.542715,3.520733e+05,0.0,1.0,36.264065,36.262805,21.246866


## IRAM 30m EMIR

In [5]:
emir = IRAM30mEMIR(linewidth)
simulator = Simulator(emir)

simulator.simulate(
    n_samples,
    samplers,
    lines,
    obstime
)

,Av,G0,Pth,angle,kappa,13c_o_j1__j0,13c_o_j2__j1,13c_o_j3__j2
0,2.847111,21.085553,3.552803e+06,0.0,1.0,6.829083,11.925423,-9.174312
1,3.040641,16.727384,8.808243e+05,0.0,1.0,2.466090,6.999750,-10.259963
2,1.317259,33.406536,2.265809e+06,0.0,1.0,-0.783804,2.974079,39.529699
3,21.889157,71.478694,2.314246e+06,0.0,1.0,39.421040,37.358304,39.375572
4,2.190706,77.452951,3.691687e+06,0.0,1.0,0.682725,4.190670,-8.518081


## Changing integration time

In [6]:
simulator.simulate(
    n_samples,
    samplers,
    lines,
    10 * obstime
)

,Av,G0,Pth,angle,kappa,13c_o_j1__j0,13c_o_j2__j1,13c_o_j3__j2
0,4.229869,26.681007,220630.823769,0.0,1.0,10.577816,12.412982,2.017352
1,3.789195,189.533693,452559.587539,0.0,1.0,5.869740,10.598447,2.437343
2,1.144148,29.433223,224806.036917,0.0,1.0,0.186957,0.247534,-2.231751
3,16.926389,354.780451,434166.693873,0.0,1.0,33.717319,35.881103,26.869483
4,1.325555,25.320594,919837.041967,0.0,1.0,0.222746,1.394906,-1.255470


## Adding a scaling parameter $\kappa$

The $\kappa$ scaling parameter is used to integrated beam dilution ($\kappa < 1$) and geometry uncertainty ($\kappa > 1$). It is always positive.

In [7]:
kappa_sampler = LogUniform(0.1**0.5, 10**0.5)

simulator.simulate(
    n_samples,
    samplers,
    lines,
    10 * obstime,
    kappa=kappa_sampler
)

,Av,G0,Pth,angle,kappa,13c_o_j1__j0,13c_o_j2__j1,13c_o_j3__j2
0,23.686477,843.422009,2.270589e+06,0.0,3.010717,138.534038,185.767947,132.078238
1,8.109191,167.207760,2.507045e+06,0.0,0.936190,16.987200,29.694781,27.264040
2,3.464913,12.616020,1.996795e+05,0.0,0.331824,2.827064,3.422085,2.893635
3,2.655290,371.130520,5.962335e+05,0.0,2.264428,2.478202,4.637277,0.899065
4,15.686875,17.287925,1.365352e+05,0.0,2.532635,78.114017,84.804080,24.215629


## Seeding

Using seeding, one can generate random yet reproducible datasets.

In [8]:
simulator.simulate(
    n_samples,
    samplers,
    lines,
    obstime,
    kappa=kappa_sampler
)

,Av,G0,Pth,angle,kappa,13c_o_j1__j0,13c_o_j2__j1,13c_o_j3__j2
0,1.762621,36.670826,1.627096e+06,0.0,2.901282,3.242177,12.028595,29.372548
1,20.872742,10.365981,1.112904e+06,0.0,0.340715,12.526270,11.554773,21.952827
2,1.796031,188.980283,2.047123e+06,0.0,0.867330,-1.521585,-7.372223,-7.909411
3,5.246656,56.475428,2.092535e+06,0.0,1.617773,20.820918,35.449467,16.807344
4,17.537479,443.548195,4.297136e+06,0.0,2.551548,93.187963,118.875414,64.150959


In [9]:
simulator.simulate(
    n_samples,
    samplers,
    lines,
    obstime,
    kappa=kappa_sampler,
    seed=0
)

,Av,G0,Pth,angle,kappa,13c_o_j1__j0,13c_o_j2__j1,13c_o_j3__j2
0,5.850707,195.788972,2.213691e+06,0.0,0.386482,7.773508,7.470687,11.525396
1,9.995208,75.019544,7.917291e+05,0.0,0.331298,5.980112,3.396069,19.040374
2,6.960285,607.499607,9.227641e+05,0.0,2.150898,33.660216,47.636578,78.493467
3,5.777155,845.912653,3.737326e+06,0.0,1.897391,15.426901,35.160349,27.747462
4,3.910605,58.463261,1.320347e+05,0.0,2.344294,21.657290,25.216067,11.163777


In [10]:
simulator.simulate(
    n_samples,
    samplers,
    lines,
    obstime,
    kappa=kappa_sampler,
    seed=0
)

,Av,G0,Pth,angle,kappa,13c_o_j1__j0,13c_o_j2__j1,13c_o_j3__j2
0,5.850707,195.788972,2.213691e+06,0.0,0.386482,7.773508,7.470687,11.525396
1,9.995208,75.019544,7.917291e+05,0.0,0.331298,5.980112,3.396069,19.040374
2,6.960285,607.499607,9.227641e+05,0.0,2.150898,33.660216,47.636578,78.493467
3,5.777155,845.912653,3.737326e+06,0.0,1.897391,15.426901,35.160349,27.747462
4,3.910605,58.463261,1.320347e+05,0.0,2.344294,21.657290,25.216067,11.163777
